In [1]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [279]:
regex = pd.read_csv(r"keywords.csv") #(r"regex.txt")

In [282]:
k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == "case"):
        p_list.append(re.compile(k[i]))
    else:
        p_list.append(re.compile(k[i], re.IGNORECASE))
print(len(p_list))

36


In [283]:
john_hsu = pd.read_csv(r"C:\users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Bert\john_hsu_mapping.csv")

In [284]:
john_hsu = john_hsu[john_hsu["NoteTXT"].notna()]

In [285]:
john_hsu = john_hsu.reset_index(drop = True)

In [28]:
l = []
for note in tqdm(john_hsu["NoteTXT"]):
    curr = []
    for p in (p_list):
        m = list(set(re.findall(p, note)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            curr.append("".join(m))
    l.append(str(curr))
    
john_hsu["regex_matches"] = l

100%|████████████████████████████████████████████████████████████████████████| 387088/387088 [1:20:58<00:00, 79.68it/s]


In [30]:
john_hsu = john_hsu[john_hsu["regex_matches"] != "[]"]

In [238]:
locations = []
for r in tqdm(john_hsu['NoteTXT']):
    curr_loc = []
    for p in p_list:
         for match in re.finditer(p ,r):
            curr_loc.append(match.span())
            
    if (curr_loc == []):
        print("STOP")
        break
    locations.append(curr_loc)
    
john_hsu["regex_location"] = locations

100%|████████████████████████████████████████████████████████████████████████████| 99334/99334 [53:51<00:00, 30.74it/s]


In [242]:
john_hsu_sequences = pd.DataFrame(columns = john_hsu.columns)

In [243]:
john_hsu = john_hsu.reset_index(drop = True)

In [247]:
for i in tqdm(range(len(john_hsu["regex_location"]))):
    curr_sequences = pd.DataFrame(columns = john_hsu.columns)
    
    #multiple matches per note, decompose list of tuples
    if (len(john_hsu["regex_location"][i]) > 1):
        for j in range(len(john_hsu["regex_location"][i])):
            #print((john_hsu["regex_location"][i][j][0], john_hsu["regex_location"][i][j][1]))
            new_regex_location = list((john_hsu["regex_location"][i][j][0], john_hsu["regex_location"][i][j][1]))
            #print(new_regex_location, "0: ", new_regex_location[0], "1: ", new_regex_location[1])
            keyword = john_hsu["NoteTXT"][i][new_regex_location[0] : new_regex_location[1]]
            
            curr_sequences = curr_sequences.append(john_hsu.loc[i])
            curr_sequences = curr_sequences.reset_index(drop = True)
            curr_sequences.at[j, "regex_matches"] = keyword
            curr_sequences.at[j, "regex_location"] = new_regex_location
    else:
        # print(john_hsu["regex_location"][i][0][0], i)
        new_regex_location = list((john_hsu["regex_location"][i][0][0], john_hsu["regex_location"][i][0][1]))
        keyword = john_hsu["NoteTXT"][i][new_regex_location[0] : new_regex_location[1]]
        curr_sequences = curr_sequences.append(john_hsu.loc[i])
        curr_sequences = curr_sequences.reset_index(drop = True)
        curr_sequences.at[0, "regex_matches"] = keyword
        curr_sequences.at[0, "regex_location"] = new_regex_location
    
    john_hsu_sequences = john_hsu_sequences.append(curr_sequences)

100%|██████████████████████████████████████████████████████████████████████████| 99334/99334 [9:30:50<00:00,  2.90it/s]


In [250]:
john_hsu_sequences = john_hsu_sequences.reset_index(drop = True)

In [261]:
for i in tqdm(range(len(john_hsu_sequences))):
    start = john_hsu_sequences["regex_location"][i][0]
    end = john_hsu_sequences["regex_location"][i][1]
    
    prune_start = False
    prune_end = False
    if (start - 600 <= 0):
        start = 0
        prune_start = True
    if (end + 600 >= len(john_hsu_sequences["NoteTXT"]) - 1):
        end = len(john_hsu_sequences["NoteTXT"]) - 1
        prune_end = True
    
    sequence = ""
    if (prune_start and prune_end):
        sequence = john_hsu_sequences["NoteTXT"][i][start : end]
    else:
        if (prune_start):
            sequence = john_hsu_sequences["NoteTXT"][i][start : end + 600]
        if (prune_end):
            sequence = john_hsu_sequences["NoteTXT"][i][start - 600: end]
    
    if (not prune_start and not prune_end):
        sequence = john_hsu_sequences["NoteTXT"][i][start - 600: end + 600]
        
    if (sequence == ""):
        print("STOP", start, end, i, prune_start, prune_end)
        break
    
    john_hsu_sequences.at[i, "regex_sent"] = sequence

100%|███████████████████████████████████████████████████████████████████████| 448818/448818 [00:17<00:00, 25417.92it/s]


In [267]:
john_hsu_sequences = john_hsu_sequences.drop(columns = ["NoteTXT_Preprocessed", "sequence_count", "sequence_length", "regex_sent_preprocessed"])

In [ ]:
notes['regex_match'] = notes['regex_match'].astype(str)
notes['regex_match'] = notes['regex_match'].str.replace("\[\]", '')

In [268]:
john_hsu['regex_sent'] = john_hsu['regex_sent'].astype(str)
john_hsu['regex_sent'] = john_hsu['regex_sent'].str.replace('[', '')
john_hsu['regex_sent'] = john_hsu['regex_sent'].str.replace(']', '')

In [269]:
john_hsu = john_hsu[john_hsu['regex_sent'].notna()]
john_hsu['regex_sent'] = john_hsu['regex_sent'].str.replace('"', "'")
john_hsu = (john_hsu.assign(regex_sent = john_hsu['regex_sent'].str.split("', '"))
         .explode('regex_sent')
         .reset_index(drop=True))

In [270]:
def note_preprocessing(note):
    note = re.sub(r'   • ', ' ; ', note)
    note = re.sub(r'[ ]{4,}', ' ', note)
    note = re.sub(r'  ', '\n', note)
    note = re.sub(r'    ', '\n\n', note)
    return note

In [273]:
for i in tqdm(range(len(john_hsu_sequences))):
    john_hsu_sequences.at[i, "regex_sent_preprocessed"] = note_preprocessing(john_hsu_sequences.at[i, "regex_sent"])

100%|███████████████████████████████████████████████████████████████████████| 448818/448818 [00:27<00:00, 16247.35it/s]


In [274]:
john_hsu_sequences["sequence_length"] = john_hsu_sequences['regex_sent_preprocessed'].astype(str).map(len)

In [277]:
john_hsu_sequences.to_csv(r"Matches/john_hsu_dataset_matches.csv", index = False)

In [278]:
john_hsu_sequences[:1000].to_csv(r"Matches/john_hsu_dataset_1000.csv", index = False)